## Change a field on both CKAN and GitHub

In [ ]:
import base64
import hashlib
import json
from datetime import datetime

import ckanapi
import requests
import yaml

### Change field on CKAN

In [ ]:
# TODO: Refactor API Key location
api_url = 'https://basedosdados.org/'
api_key = ''

In [ ]:
# TODO: Refactor RemoteCKAN scope
ckan = RemoteCKAN(api_url, api_key)

In [ ]:
def get_ckan_table(dataset_id: str, table_id: str) -> dict:
    dataset = ckan.action.package_show(id=dataset_id)
    table = filter(lambda r: r['name'] == table_id, dataset['resources'])
    table = list(table)[0]
    
    return table

table = get_ckan_table('br-ana-reservatorios', 'sin')

In [ ]:
def update_time(table: dict) -> dict:
    '''Update table_config.yaml last updated time'''
    if not table.get('last_updated'):
        table['last_updated'] = {
            'data': None,
            'metadata': None,
            'release': None,
        }
    elif not table['last_updated'].get('data'):
        table['last_updated']['data'] = None
    elif not table['last_updated'].get('metadata'):
        table['last_updated']['metadata'] = None
    elif not table['last_updated'].get('release'):
        table['last_updated']['release'] = None
    
    # TODO: Validate the datetime format
    table['last_updated']['data'] = str(datetime.now())
    
    return table

table_mod = update_time(table)

In [ ]:
# TODO: Test this function in a local or staging environment
def set_ckan_table(dataset_id: str, table_id: str, table: dict) -> bool:
    response = ckan.action.resource_update(**table)
    return response['success']

set_ckan_table('br-ana-reservatorios', 'sin', table_mod)

### Change field on GitHub

In [ ]:
# TODO: Refactor the authorization header

In [ ]:
access_token = 'fake_token'

headers = {
    'Accept': 'application/vnd.github.v3+json',
    'Authorization': f'token {access_token}',
}

In [ ]:
def get_github_table(database: str, table: str) -> [str, dict]:
    '''Get basedosdados table_config.yaml with dict type'''
    url = 'https://api.github.com/repos/basedosdados/mais'
    url += f'/contents/bases/{database}/{table}/table_config.yaml'
    
    response = requests.get(url, headers=headers).json()
    
    sha = response['sha']
    
    result = base64.b64decode(response['content'])
    result = result.decode('utf-8')
    result = yaml.safe_load(result)
    
    return sha, result

sha, table = get_github_table('br_ana_reservatorios', 'sin')

In [ ]:
def update_time(table: dict) -> dict:
    '''Update table_config.yaml last updated time'''
    if not table.get('last_updated'):
        table['last_updated'] = {
            'data': None,
            'metadata': None,
            'release': None,
        }
    elif not table['last_updated'].get('data'):
        table['last_updated']['data'] = None
    elif not table['last_updated'].get('metadata'):
        table['last_updated']['metadata'] = None
    elif not table['last_updated'].get('release'):
        table['last_updated']['release'] = None
    
    # TODO: Validate the datetime format
    table['last_updated']['data'] = str(datetime.now())
    
    return table

table_mod = update_time(table)

In [ ]:
# TODO: Bypass 'At least 1 approving review is required by reviewers with write access'
# TODO: Format table_config.yaml. Maybe use regex?

In [ ]:
def set_github_table(database: str, table: str, sha: str, content: dict) -> bool:
    '''Set basedosdados table_config.yaml'''
    url = 'https://api.github.com/repos/basedosdados/mais'
    url += f'/contents/bases/{database}/{table}/table_config.yaml'
    
    message = f'feat: prefect update table {database}.{table}'
    
    content = yaml.safe_dump(content)
    content = content.encode('utf-8')
    content = base64.b64encode(content)
    content = content.decode('utf-8')
    
    data = dict(message=message, content=content, sha=sha)
    data = json.dumps(data)
    
    response = requests.put(url, headers=headers, data=data).json()

    return response

k = set_github_table('br_ana_reservatorios', 'sin', sha, table_mod)